# DEEPX Tutorial 05 - DX Compile workflow


In Tutorial 5, you will practice compiling classification, object detection, and segmentation models using the DX-Compiler. We will also review the guide for troubleshooting problems during compilation.

For more details, refer to the DX-COM user guide 👉 [Download](https://developer.deepx.ai/?files=MjUxNQ==)

>This tutorial is based on dx-all-suite v2.0.0, released in September 2025.

## Compiling Image Classification Model
1. Export PyTorch → ONNX
2. JSON configuration for Input/Pre-processing/Calibration
3. Compile with DX-Compiler and verify .dxnn

### 1. Export PyTorch → ONNX

In [ ]:
# Install pytorch & onnx
!pip install --quiet torch torchvision onnx

In [ ]:
# Move to "dx-tutorials/dx-all-suite/dx-compiler/dx_com"
import os
root_path = os.environ.get('ROOT_PATH')
%cd $root_path/dx-all-suite/dx-compiler/dx_com

Export Pytorch based MobileNetV2 model to ONNX

In [ ]:
import torch, torchvision

model = torchvision.models.mobilenet_v2(weights=torchvision.models.MobileNet_V2_Weights.DEFAULT)
model.eval()
dummy = torch.randn(1, 3, 224, 224)

onnx_path = "MobilenetV2.onnx"
torch.onnx.export(model, dummy, onnx_path, input_names=["input"], output_names=["output"], opset_version=13)
print("✅ Save ONNX:", onnx_path)

### 2. JSON configuration for Input/Pre-processing/Calibration
Generate a configuration file for Input/Pre-processing/Calibration of MobilenetV2

In [ ]:
import json, os

# 캘리브레이션 이미지 폴더 예시: ./dx_com/calibration_dataset
cfg = {
  "inputs": {"input": [1,3,224,224]},
  "calibration_num": 10,
  "calibration_method": "ema",
  "default_loader": {
    "dataset_path": "./calibration_dataset",
    "file_extensions": ["jpg","jpeg","png"],
    "preprocessings": [
      {"convertColor": {"form": "BGR2RGB"}},
      {"resize": {"width": 224, "height": 224}},
      {"div": {"x": 255.0}},
      {"normalize": {"mean": [0.485,0.456,0.406], "std": [0.229,0.224,0.225]}}
    ]
  }
}
with open("MobilenetV2.json","w") as f:
    json.dump(cfg, f, indent=2)
print("✅ JSON 저장: MobilenetV2.json")
os.system("sed -n '1,120p' MobilenetV2.json")

%cp MobilenetV2.json $root_path/

### 3. Compile with DX-Compiler and verify .dxnn
Compile with dx_com to generate .dxnn

In [ ]:
!./dx_com/dx_com -m MobilenetV2.onnx -c MobilenetV2.json -o ./

### 4. Update JSON configuration to fix the following error:
> ConfigInputError: Config shape [1, 3, 224, 224] does not match preprocessed data shape [1, 224, 224, 3]

In [ ]:
import json, os

# 캘리브레이션 이미지 폴더 예시: ./dx_com/calibration_dataset
cfg = {
  "inputs": {"input": [1,3,224,224]},
  "calibration_num": 10,
  "calibration_method": "ema",
  "default_loader": {
    "dataset_path": "./calibration_dataset",
    "file_extensions": ["jpg","jpeg","png"],
    "preprocessings": [
      {"convertColor": {"form": "BGR2RGB"}},
      {"resize": {"width": 224, "height": 224}},
      {"div": {"x": 255.0}},
      {"normalize": {"mean": [0.485,0.456,0.406], "std": [0.229,0.224,0.225]}},
      {"transpose": {"axis": [2,0,1]}},
      {"expandDim": {"axis": 0}}
    ]
  }
}
with open("MobilenetV2.json","w") as f:
    json.dump(cfg, f, indent=2)
print("✅ JSON 저장: MobilenetV2.json")
os.system("sed -n '1,120p' MobilenetV2.json")

%cp MobilenetV2.json $root_path/

### 5. Re-compile with DX-Compiler and verify .dxnn
Compile with dx_com to generate .dxnn and verify it with run_model:

In [ ]:
!./dx_com/dx_com -m MobilenetV2.onnx -c MobilenetV2.json -o ./
!ls | grep MobilenetV2.dxnn

In [ ]:
!run_model -m MobilenetV2.dxnn

## Compiling Object Detection Model
1. Export PyTorch → ONNX
2. JSON configuration for Input/Pre-processing/Calibration
3. Compile with DX-Compiler and verify .dxnn

In [ ]:
!pip install onnx onnxscript ultralytics

In [27]:
# Move to "dx-tutorials/dx-all-suite/dx-compiler/dx_com"
import os
root_path = os.environ.get('ROOT_PATH')
%cd $root_path/dx-all-suite/dx-compiler/dx_com

/home/max/Works/dx-tutorials/dx-all-suite/workspace/release/dx_com/dx_com_M1_v2.0.0


In [28]:
!wget https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-s.pt

--2025-10-29 21:23:07--  https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-s.pt
Resolving github.com (github.com)... 20.200.245.247
Connecting to github.com (github.com)|20.200.245.247|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/759338070/5efccc50-06db-4a1f-bf78-db9fcce17d09?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-10-29T13%3A08%3A00Z&rscd=attachment%3B+filename%3Dyolov9-s.pt&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-10-29T12%3A07%3A18Z&ske=2025-10-29T13%3A08%3A00Z&sks=b&skv=2018-11-09&sig=5CAPFguDYI8mi3UPAcFW4ReYHYr5jqkdsen4mO6CTII%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2MTc0MjM4NywibmJmIjoxNzYxNzQwNTg3LCJwYXRoIjoicmVsZWFzZWFzc2V0cHJvZHVjdGlvbi5ibG9iLmNvcm

In [29]:
import torch
import onnx
from ultralytics import YOLO

model = YOLO("yolov9-s.pt") 

# Export to ONNX, fixed input size 1x3x640x640, simplified graph
model.export(
    format="onnx",
    imgsz=640,          # or (640, 640)
    opset=13,           # 12–13 is the most compatible range
    simplify=True,
    dynamic=False       # fixed shape (batch=1, H=W=640)
)

Ultralytics 8.3.213 🚀 Python-3.12.3 torch-2.9.0+cu128 CPU (AMD Ryzen 5 9600X 6-Core Processor)


Fusing layers... 
Model summary: 658 layers, 9659168 parameters, 0 gradients, 39.1 GFLOPs



PyTorch: starting from 'yolov9-s.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) ((), ()) (19.5 MB)

ONNX: starting export with onnx 1.19.1 opset 13...


W1029 21:23:22.850000 58963 torch/onnx/_internal/exporter/_compat.py:114] Setting ONNX exporter to use operator set version 18 because the requested opset_version 13 is a lower version than we have implementations for. Automatic version conversion will be performed, which may not be successful at converting to the requested version. If version conversion is unsuccessful, the opset version of the exported model will be kept at 18. Please consider setting opset_version >=18 to leverage latest ONNX features
The model version conversion is not supported by the onnxscript version converter and fallback is enabled. The model will be converted using the onnx C API (target version: 13).
Failed to convert the model to the target version 13 using the ONNX C API. The model was not modified
Traceback (most recent call last):
  File "/home/max/Works/dx-tutorials/.venv/lib/python3.12/site-packages/onnxscript/version_converter/__init__.py", line 127, in call
    converted_proto = _c_api_utils.call_on

ONNX: slimming with onnxslim 0.1.72...
ONNX: export success ✅ 3.5s, saved as 'yolov9-s.onnx' (37.1 MB)

Export complete (4.2s)
Results saved to /home/max/Works/dx-tutorials/dx-all-suite/workspace/release/dx_com/dx_com_M1_v2.0.0
Predict:         yolo predict task=detect model=yolov9-s.onnx imgsz=640  
Validate:        yolo val task=detect model=yolov9-s.onnx imgsz=640 data=None  
Visualize:       https://netron.app


'yolov9-s.onnx'

In [23]:
!cp yolov9-s.onnx ~/yolov9-s-non-simple.onnx

In [32]:
import json, os

# 캘리브레이션 이미지 폴더 예시: ./dx_com/calibration_dataset
cfg = {
  "inputs": {    "images": [      1,      3,      640,      640    ]  },
  "calibration_num": 100,
  "calibration_method": "ema",
  "default_loader": {
    "dataset_path": "./calibration_dataset",
    "file_extensions": [      "jpeg",      "jpg",      "png",      "JPEG"    ],
    "preprocessings": [
      {
        "resize": {          "mode": "pad",          "size": 640,          "pad_location": "edge",
          "pad_value": [            114,            114,            114          ]        }
      },
      {
        "div": {          "x": 255        }
      },
      {
        "convertColor": {          "form": "BGR2RGB"        }
      },
      {
        "transpose": {          "axis": [            2,            0,            1          ]        }
      },
      {        "expandDim": {          "axis": 0        }      }
    ]
  }
}

with open("yolov9-s.json","w") as f:
    json.dump(cfg, f, indent=2)
print("✅ JSON 저장: yolov9-s.json")
os.system("sed -n '1,120p' yolov9-s.json")

✅ JSON 저장: yolov9-s.json
{
  "inputs": {
    "images": [
      1,
      3,
      640,
      640
    ]
  },
  "calibration_num": 100,
  "calibration_method": "ema",
  "default_loader": {
    "dataset_path": "./calibration_dataset",
    "file_extensions": [
      "jpeg",
      "jpg",
      "png",
      "JPEG"
    ],
    "preprocessings": [
      {
        "resize": {
          "mode": "pad",
          "size": 640,
          "pad_location": "edge",
          "pad_value": [
            114,
            114,
            114
          ]
        }
      },
      {
        "div": {
          "x": 255
        }
      },
      {
        "convertColor": {
          "form": "BGR2RGB"
        }
      },
      {
        "transpose": {
          "axis": [
            2,
            0,
            1
          ]
        }
      },
      {
        "expandDim": {
          "axis": 0
        }
      }
    ]
  }
}

0

In [33]:
!./dx_com/dx_com -m yolov9-s.onnx -c yolov9-s.json -o ./

Compiling Model :  81%|███████████████▍   | 0.8/1.0 [00:19<00:04, 22.03s/model ]Nuitka: A segmentation fault has occurred. This is highly unusual and can
have multiple reasons. Please check https://nuitka.net/info/segfault.html
for solutions.
